In [1]:
import requests
import json
import pandas as pd
import altair as alt

In [2]:
def API_call(series_ids, start_year, end_year):
    '''
    Calls the BLS API to return data. Returns a DataFrame with the combined results.
    '''
    
    ### MAKE SURE TO SET YOUR API KEY BELOW.
    api_key = 'XXXXX'
    
    # Build message to send to API.
    headers = {'Content-type': 'application/json'}
    data = json.dumps({"seriesid": series_ids,"startyear":start_year, "endyear":end_year, "registrationkey":api_key})
    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/?registrationkey=9026b9bdf3e24f269aa6c29da82a189d', data=data, headers=headers)

    # Parse results.
    j = json.loads(p.text)
    dfs = []
    for x in range(0,len(j["Results"]['series'])):
        t_df = pd.DataFrame(j["Results"]['series'][x]['data'])
        t_df['series'] = j["Results"]['series'][x]['seriesID']
        dfs.append(t_df)
    df = pd.concat(dfs)
    
    return df

### CPI Data

In [14]:
# Import list of targeted CPI-U cateogires.
targets = pd.read_excel("CPI_Category_Tree_Final - REVISED.xlsx", header=0)
targets

,Category Name,Revised Name,Revised Name with CPI,Series ID,Level,Parent Series ID,Leaf
0,CPI - All items,All items,CPI - All items,CUSR0000SA0,0,NaN,0
1,CPI - Food and Beverages,Food & Beverages,CPI - Food & Beverages,CUSR0000SAF,1,CUSR0000SA0,0
2,CPI - Food at home,Food at home,CPI - Food at home,CUSR0000SAF11,2,CUSR0000SAF,0
3,CPI - Cereals and bakery products,Cereals & bakery products,CPI - Cereals & bakery products,CUSR0000SAF111,3,CUSR0000SAF11,1
4,"CPI - Meats, poultry, fish, and eggs","Meats, poultry, fish, & eggs","CPI - Meats, poultry, fish, & eggs",CUSR0000SAF112,3,CUSR0000SAF11,1
...,...,...,...,...,...,...,...
141,CPI - Legal services,Legal services,CPI - Legal services,CUSR0000SEGD01,3,CUSR0000SEGD,1
142,CPI - Funeral expenses,Funeral expenses,CPI - Funeral expenses,CUSR0000SEGD02,3,CUSR0000SEGD,1
143,CPI - Laundry and dry cleaning services,Laundry & dry cleaning services,CPI - Laundry & dry cleaning services,CUSR0000SEGD03,3,CUSR0000SEGD,1
144,CPI* - Apparel services other than laundry and...,Other apparel services,CPI* - Other apparel services,CUUR0000SEGD04,3,CUSR0000SEGD,1


In [4]:
# Split query inputs into chunks that fit in the API limit.
year_ranges = [(x,x+19) for x in range(1970,2021,20)]
series_ids = list(targets['Series ID'])
series_id_groups = [series_ids[0:50],series_ids[50:100],series_ids[100:150]]

# Send API requests and combine into a single DataFrame
df = pd.concat([API_call(series_id_groups[y], year_ranges[x][0], year_ranges[x][1]) for x in range(0,3) for y in range(0,3)])

In [15]:
# Merge other data into results from API pull.
series_names = targets.set_index('Series ID').to_dict()['Revised Name with CPI']
df['Category'] = df['series'].map(series_names)
parent_IDs = targets.set_index('Series ID').to_dict()['Parent Series ID']
df['Parent Series ID'] = df['series'].map(parent_IDs)
levels = targets.set_index('Series ID').to_dict()['Level']
df['Level'] = df['series'].map(levels)
leaves = targets.set_index('Series ID').to_dict()['Leaf']
df['Leaf'] = df['series'].map(leaves)

# Convert month and year to a datetime column.
df['date'] = pd.to_datetime(df.year.astype(str) + '/' + df.period.str[1:] + '/01')

# Make sure values are stored as numbers and not as strings.
df.value = df.value.astype(float)

### Non-CPI Data

In [16]:
# Import list of targeted non-CPI cateogires.
targets = pd.read_excel("Other_BLS_Data_Final - REVISED.xlsx", header=0)
targets

,Category Name,Revised Category Name,Series ID,Type,Bucket
0,Earnings - All People,E - All,LEU0252881500,Earnings,Total
1,Earnings - Men,E - Men,LEU0252881800,Earnings,By Gender
2,Earnings - Women,E - Women,LEU0252882700,Earnings,By Gender
3,Earnings - White People,E - White People,LEU0252883600,Earnings,By Race
4,Earnings - White Men,E - White Men,LEU0252883900,Earnings,By Race and Gender
5,Earnings - White Women,E - White Women,LEU0252884200,Earnings,By Race and Gender
6,Earnings - Black People,E - Black People,LEU0252884500,Earnings,By Race
7,Earnings - Black Men,E - Black Men,LEU0252884800,Earnings,By Race and Gender
8,Earnings - Black Women,E - Black Women,LEU0252885100,Earnings,By Race and Gender
9,Earnings - Asian People,E - Asian People,LEU0254468400,Earnings,By Race


In [7]:
# Split query inputs into chunks that fit in the API limit.
year_ranges = [(x,x+19) for x in range(1970,2021,20)]
series_ids = list(targets['Series ID'])
series_id_groups = [series_ids[0:50]]

# Send API requests and combine into a single DataFrame
df2 = pd.concat([API_call(series_id_groups[y], year_ranges[x][0], year_ranges[x][1]) for x in range(0,3) for y in range(0,1)])

In [17]:
# Merge other data into results from API pull.
series_names = targets.set_index('Series ID').to_dict()['Revised Category Name']
df2['Category'] = df2['series'].map(series_names)
types = targets.set_index('Series ID').to_dict()['Type']
df2['Type'] = df2['series'].map(types)
buckets = targets.set_index('Series ID').to_dict()['Bucket']
df2['Bucket'] = df2['series'].map(buckets)

# Convert quarter names to months.
df2['period'].replace(to_replace = ['Q01', 'Q02', 'Q03', 'Q04'], value = ['M03', 'M06', 'M09', 'M12'], inplace = True)

# Convert month and year to a datetime column.
df2['date'] = pd.to_datetime(df2.year.astype(str) + '/' + df2.period.str[1:] + '/01')

# Make sure values are stored as numbers and not as strings.
df2.value = df2.value.astype(float)

In [21]:
# Get stock data.
stocks = pd.read_excel("Stock_Markets_Data - REVISED.xlsx", header=0)

# User revised category names
stocks['Category'] = stocks['Revised Category']

# Adjust format of stocks data to align with other data.
stocks = stocks.set_index('date').groupby('Category').resample('M').mean().reset_index()
stocks['date'] = stocks['date'] + pd.offsets.MonthBegin(-1)
stocks['year'] = stocks['date'].dt.year

# Add bucket labels
stocks['Type'] = 'Stocks'

In [22]:
combo_df = pd.concat([df, df2, stocks])

In [24]:
# Save DataFrame as pickle.
combo_df.year = combo_df.year.astype(int)
combo_df.to_pickle("combined_data.pkl")

In [23]:
# Check data
combo_df

,year,period,periodName,value,footnotes,series,latest,Category,Parent Series ID,Level,Leaf,date,Type,Bucket
0,1989,M12,December,126.300000,[{}],CUSR0000SA0,NaN,CPI - All items,NaN,0.0,0.0,1989-12-01,NaN,NaN
1,1989,M11,November,125.900000,[{}],CUSR0000SA0,NaN,CPI - All items,NaN,0.0,0.0,1989-11-01,NaN,NaN
2,1989,M10,October,125.400000,[{}],CUSR0000SA0,NaN,CPI - All items,NaN,0.0,0.0,1989-10-01,NaN,NaN
3,1989,M09,September,124.800000,[{}],CUSR0000SA0,NaN,CPI - All items,NaN,0.0,0.0,1989-09-01,NaN,NaN
4,1989,M08,August,124.500000,[{}],CUSR0000SA0,NaN,CPI - All items,NaN,0.0,0.0,1989-08-01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1872,2022,NaN,NaN,4435.980526,NaN,NaN,NaN,S - S&P,NaN,NaN,NaN,2022-02-01,Stocks,NaN
1873,2022,NaN,NaN,4391.265217,NaN,NaN,NaN,S - S&P,NaN,NaN,NaN,2022-03-01,Stocks,NaN
1874,2022,NaN,NaN,4391.296000,NaN,NaN,NaN,S - S&P,NaN,NaN,NaN,2022-04-01,Stocks,NaN
1875,2022,NaN,NaN,4040.360000,NaN,NaN,NaN,S - S&P,NaN,NaN,NaN,2022-05-01,Stocks,NaN
